In [1]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader(r"D:\Generative AI for Legal Aids\Prototype 2\data\bns.pdf")
docs = pdf_loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'D:\\Generative AI for Legal Aids\\Prototype 2\\data\\bns.pdf', 'page': 0}, page_content='1 \n THE BHARATIYA NYAYA SANHITA, 2023  \n________  \nARRANGEMENT OF SECTIONS  \n________  \nCHAPTER I  \nPRELIMINARY  \nSECTIONS  \n1. Short title, commencement and application.  \n2. Definitions.  \n3. General explanations.  \nCHAPTER II  \nOF PUNISHMENTS  \n4. Punishments.  \n5. Commutation of sentence.  \n6. Fractions of terms of punishment.  \n7. Sentence may be (in certain cases of imprisonment) wholly or partly rigorous or simple.  \n8. Amount of fine, liability in default of payment of fine, etc.  \n9. Limit of punishment of offence mad e up of several offences.  \n10. Punishment of person guilty of one of several offences, judgment stating that it is doubtful of \nwhich.  \n11. Solitary confinement.  \n12. Limit of solitary confinement.  \n13. Enhanced punishment for certain offences after previous co nviction.  \nCHAPTER III  \nGENERAL EXCEPTIONS  \n14. Act 

In [3]:
print(len(documents))

526


In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

ollama_emb = OllamaEmbeddings(model="llama2:7b")

db = FAISS.from_documents(documents=documents,   embedding=ollama_emb)

C:\Users\pulki\AppData\Local\Temp\ipykernel_3536\1224078049.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_emb = OllamaEmbeddings(model="llama2:7b")


In [5]:
query="A person hitted my friend by a rod then tell me which BNS section will be applied"
result=db.similarity_search(query)
result[0].page_content

'punished with imprisonment for a term which shall not be less than five years but which may extend to \nimprisonment for life, and shall also be liable to fine.  \n(5) Any person wh o is a member of an organisation which is involved in terrorist act, shall be \npunished with imprisonment for a term which may extend to imprisonment for life, and shall also be \nliable to fine.  \n(6) Whoever voluntarily harbours or conceals, or attempts to ha rbour or conceal any person knowing \nthat such person has committed a terrorist act shall be punished with imprisonment for a term which shall \nnot be less than three years but which may extend to imprisonment for life, and shall also be liable to \nfine: \nProv ided that this sub -section shall not apply to any case in which the harbour or concealment is by \nthe spouse of the offender.  \n(7) Whoever knowingly possesses any property derived or obtained from commission of any terrorist'

In [6]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2:7b")
llm

C:\Users\pulki\AppData\Local\Temp\ipykernel_3536\2136070663.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama2:7b")


Ollama(model='llama2:7b')

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
You are n Expert Legal Advisor and you have to answer Bhartiya Nyay Sanhita Sections
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [9]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028A04E4E5F0>, search_kwargs={})

In [10]:
db.save_local("faiss_index")

In [11]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [12]:
response=retrieval_chain.invoke({"input":"A man hit my friend with a rod. tell me BNS Section"})
response['answer']

'Based on the context provided, the relevant Bhartiya Nyay Sanhita (BNS) section for the crime of hitting someone with a rod is Section 307. This section states that whoever assaults or threatens to assault any person with a weapon, such as a rod, with the intention of causing physical harm, shall be punishable with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine and to forfeiture of any property used or intended to be used in committing the offence.'

In [13]:
print(response['answer'])

Based on the context provided, the relevant Bhartiya Nyay Sanhita (BNS) section for the crime of hitting someone with a rod is Section 307. This section states that whoever assaults or threatens to assault any person with a weapon, such as a rod, with the intention of causing physical harm, shall be punishable with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine and to forfeiture of any property used or intended to be used in committing the offence.
